In [7]:
Bf_z = range(100,1100,100)
#arcpy.management.SetRasterProperties("Troll_T1_Mos2.tif", "THEMATIC", None, None, 1 3, None, None)

In [8]:
#DEfine Project Path
import arcpy
#aprx_path = r'X:\Projects\2019\ENV.533.aprx'
#gdb_path = r'X:\shared\ENV_default.gdb'
from arcpy import env
env.parallelProcessingFactor = "0"
env.overwriteOutput = True
env.addOutputsToMap = False
env.cellSize = 0.2
import pandas as pd
aprx = arcpy.mp.ArcGISProject('CURRENT') # or 'CURRENT' in live py console
aprx.defaultGeodatabase
import pandas as pd
import numpy as np

In [9]:
#Model for Land Transformation around Reservoir 
def Model1(T1_Inp, T3_Inp, T1_Buffer, OP_XLSX, T3_Buffer,Mag):  # Model 1
    env.parallelProcessingFactor = "0"
    env.overwriteOutput = True
    env.addOutputsToMap = False
    env.cellSize = 0.2
    env.snapRaster=T1_Inp
    env.extent = T1_Inp
    env.mask = T1_Inp
   # Check out any necessary licenses.
    arcpy.CheckOutExtension("spatial")
    arcpy.ImportToolbox(r"c:\program files\arcgis\pro\Resources\ArcToolbox\toolboxes\Conversion Tools.tbx")
    Mag_buff = "NVE\\Mag_buff"
    Magasin = Mag
    # Process: Erase (Erase) (analysis)
    Output_Feature_Class = "Mag_buff_Erase"
    arcpy.analysis.Erase(in_features=Mag_buff, erase_features=Magasin, out_feature_class=Output_Feature_Class, cluster_tolerance="")
    print("Earse Done")
    # Process: Extract by Mask (Extract by Mask) (sa)
    Extract_by_Mask = T3_Buffer
    T3_Buffer = arcpy.sa.ExtractByMask(in_raster=T3_Inp, in_mask_data=Output_Feature_Class)
    T3_Buffer.save(Extract_by_Mask)
    print("Mask 3 Done",flush = True)

    # Process: Table Select (2) (Table Select) (analysis)
    Output_Table_2_ = "Extract_Tons2_TableSelect"
    arcpy.analysis.TableSelect(in_table=T3_Buffer, out_table=Output_Table_2_, where_clause="")

    # Process: Extract by Mask (2) (Extract by Mask) (sa)
    Extract_by_Mask_2_ = T1_Buffer
    with arcpy.EnvManager(extent=T3_Buffer, mask=T3_Buffer):
        T1_Buffer = arcpy.sa.ExtractByMask(in_raster=T1_Inp, in_mask_data=Output_Feature_Class)
        T1_Buffer.save(Extract_by_Mask_2_)
    print("Mask 1 Done",flush = True)


    # Process: Table Select (Table Select) (analysis)
    Output_Table = "Extract_Tons1_TableSelect"
    arcpy.analysis.TableSelect(in_table=T1_Buffer, out_table=Output_Table, where_clause="")

    # Process: Calculate Field (Calculate Field) (management)
    Extract_Trol2_TableSelect = arcpy.management.CalculateField(in_table=Output_Table, field="Area_KM2_T1", expression="!COUNT!*0.2*0.2/1000000", expression_type="PYTHON3", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (Join Field) (management)
    Extract_Mos_1_TableSelect_2_ = arcpy.management.JoinField(in_data=Output_Table_2_, in_field="VALUE", join_table=Extract_Trol2_TableSelect, join_field="VALUE", fields=["Area_KM2_T1"])[0]

    # Process: Calculate Field (2) (Calculate Field) (management)
    Extract_Mos_1_TableSelect = arcpy.management.CalculateField(in_table=Extract_Mos_1_TableSelect_2_, field="Area_KM2_T3", expression="!COUNT!*0.2*0.2/1000000", expression_type="PYTHON3", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Table To Excel (Table To Excel) (conversion)
    arcpy.conversion.TableToExcel(Input_Table=Extract_Mos_1_TableSelect, Output_Excel_File=OP_XLSX, Use_field_alias_as_column_header="NAME", Use_domain_and_subtype_description="CODE")

In [10]:
#Running Model for LT around Reservoirs 
T1_Inp = "MOS_T1_v2.tif"
T3_Inp = "MOS_T3_v2.tif"
T1_Buffer= "Kaldevatn_T1"
OP_XLSX = r"D:\mskenawi\one drive\OneDrive - NTNU\PhD\LUHP\Summary\Summary of Data\LT_Buf_Res\Kaldevatn_LT.xlsx"
T3_Buffer = "Kaldevatn_T3"
Mag = "Saur_res"
Model1(T1_Inp,T3_Inp,T1_Buffer,OP_XLSX,T3_Buffer,Mag)

Earse Done
Mask 3 Done
Mask 1 Done


In [5]:
#Input for Land OCCUPATION OF RESERVOIRS (MAKE SURE TO CHANGE OP VARIABLE)
Rst_T1 = "Brokke_T1_v3.tif"
Clp = "Brokke_Res"
OP = "Brokke"
Resolution = float(arcpy.GetRasterProperties_management(Rst_T1, "CELLSIZEX").getOutput(0))
print("Res is", Resolution)

Res is 0.2


In [6]:
#Direct Impact Running
LO = list()
fields = ['OID@','magasinNavn']# OID@ is ObjectID token (see search cursor help)
# clear any existing selections
arcpy.SelectLayerByAttribute_management(Clp, "CLEAR_SELECTION")
#Select Feature 
cursor = arcpy.da.SearchCursor(Clp, fields)
for FID,N in  (cursor): # unpack each row by assigning to field name variables
        print(FID)
        print(N)
#Remove Spaces from the name becasue it makes Error
        N_S = N.replace(" ","")
        N_S = N_S.replace("-","_")
        arcpy.management.SelectLayerByAttribute(Clp, "NEW_SELECTION", f"OBJECTID = {FID}", None)
#Get the Area for each Reservoir  
        do = arcpy.sa.ExtractByMask(Rst_T1, Clp)
        print (rf"Direct_LO_{N_S}")
        do.save(rf"Direct_LO_{N_S}") 
        arcpy.management.BuildRasterAttributeTable(rf"Direct_LO_{N_S}", "Overwrite", "NONE")
        arcpy.management.CalculateField(rf"Direct_LO_{N_S}", "Res_Name", f'"{N_S}"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
        df= pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(rf"Direct_LO_{N_S}",('Value', 'Count','Res_Name')))
        LO.append(df)
        

1
Bykil
Direct_LO_Bykil


In [7]:
#Saving Direct Impact to Excel file 
Area_D = pd.concat(LO)
Area_D["Area_KM2"] = Area_D["Count"]*0.2*0.2/1000000
Area_D.to_excel(rf'D:\mskenawi\one drive\OneDrive - NTNU\PhD\LUHP\Summary\Summary of Data\Direct_LU_Res\{OP}.xlsx')

In [3]:
#Define : Compute change raster, Mag_Feat, Mask
#NAME OF COMPUTE  CHANGE RASTER
Rs_ch = "AnaSira_ComputeChange.crf"
Clp = "Ana_Res"
Msk = "AnaSira_T1"
OP_N = "AnaSira"

In [4]:
#Import Class name
arcpy.analysis.TableSelect(Rs_ch,rf"CC_Table", '')
df_CC= arcpy.da.FeatureClassToNumPyArray("CC_Table",('Class_From', 'Class_To','Area'))

In [5]:
#Buffers 

Area_bf = []
for i in Bf_z:
    arcpy.analysis.Buffer(Clp, 
                            f"Buf", f"{i} Meters", "FULL", "ROUND", "NONE", None, "GEODESIC")
#Earse Magasins
    arcpy.analysis.Erase("Buf", Clp, "tt_Erase", None)
#Clip from the big Buffer
    arcpy.analysis.Clip("tt_Erase", r"NVE\Mag_buff", "tt_Clip", None)
#Get the Area for each interval 
    with arcpy.EnvManager(mask=Msk):
        out_raster2 = arcpy.sa.ExtractByMask(Rs_ch, "tt_Clip")
        out_raster2.save(rf"CC_{OP_N}_{i}")
        arcpy.management.Delete("out_raster2;Buf;tt_Erase;tt_Clip", '')
    print(rf"CC_{OP_N}_{i}")
    #arcpy.management.BuildRasterAttributeTable(f"CC_{i}", "Overwrite", "NONE")
    #arcpy.analysis.TableSelect("out_raster",rf"CC_Troll_{i}", '')
    #df_CC= pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(rf"CC_Troll_{i}",('Class_From', 'Class_To','Area')))
    #Area_bf.append(df_CC)
    
    


CC_AnaSira_100
CC_AnaSira_200
CC_AnaSira_300
CC_AnaSira_400
CC_AnaSira_500
CC_AnaSira_600
CC_AnaSira_700
CC_AnaSira_800
CC_AnaSira_900
CC_AnaSira_1000


In [6]:
Area_bf = []
R1 = arcpy.ListRasters(rf"CC_{OP_N}_*")
print(R1)
for i in R1:
    arcpy.management.BuildRasterAttributeTable(i, "Overwrite", "NONE")
    arcpy.management.JoinField(i, "Value", Rs_ch, "Value", "Class_name;Class_From;Class_To")
for i in R1:
    df_CC= pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(i,('Class_From', 'Class_To','Count')))
    Area_bf.append(df_CC) 

    

['CC_AnaSira_100', 'CC_AnaSira_200', 'CC_AnaSira_300', 'CC_AnaSira_400', 'CC_AnaSira_500', 'CC_AnaSira_600', 'CC_AnaSira_700', 'CC_AnaSira_800', 'CC_AnaSira_900', 'CC_AnaSira_1000']


In [ ]:
#Just for reWriting my List DONT USE
Area_bf = []
for i in R1:
   df_CC= pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(i,('Class_From', 'Class_To','Count')))
   Area_bf.append(df_CC) 
    
#Area_bf = pd.concat(Area_bf)
#Area_bf.dtypes


In [8]:
for i, j  in enumerate(Area_bf):
    print(i)
    print(j)
    j['Buffer_Zone'] = Bf_z[i]

0
   Class_From Class_To       Count
0           0        1   1624266.0
1           0        2    305378.0
2           0        4    212948.0
3           0        5    711800.0
4           1        0   2747043.0
5           1        2   3748623.0
6           1        4   1189820.0
7           1        5   8189301.0
8           2        0    681316.0
9           2        1   3384892.0
10          2        4     79291.0
11          2        5   1203615.0
12          4        0   1431466.0
13          4        1   2576061.0
14          4        2    131289.0
15          4        5    654411.0
16          5        0   3649243.0
17          5        1  23943387.0
18          5        2   2026148.0
19          5        4    854941.0
20       Same     Same  93581157.0
1
   Class_From Class_To        Count
0           0        1    2491482.0
1           0        2     312065.0
2           0        4     493779.0
3           0        5     951033.0
4           1        0    4067281.0
5         

In [9]:
for i in Area_bf : 
    i["Area_KM2"] = i['Count']*.2*.2/1000000

In [10]:
Area_BF = pd.concat(Area_bf)
Area_BF

,Class_From,Class_To,Count,Buffer_Zone,Area_KM2
0,0,1,1624266.0,100,0.064971
1,0,2,305378.0,100,0.012215
2,0,4,212948.0,100,0.008518
3,0,5,711800.0,100,0.028472
4,1,0,2747043.0,100,0.109882
...,...,...,...,...,...
16,5,0,12414510.0,1000,0.496580
17,5,1,256735591.0,1000,10.269424
18,5,2,4140350.0,1000,0.165614
19,5,4,8754825.0,1000,0.350193


In [11]:
Area_BF.dtypes
#Filtering Deforestiation : 1 to 0,2,4,5 
Area_Def =Area_BF[(Area_BF["Class_From"].isin(["1","Forest"]) & (Area_BF["Class_To"].isin(["0","2","4","5",'Urban','Water','Cultivated','Bare'])))] 
#Filtering Urbanization : All to 0 
Area_Urb =Area_BF[Area_BF["Class_To"].isin(["0","Urban"]) & (Area_BF["Class_From"].isin(["1","2","4","5",'Forest','Water','Cultivated','Bare']))] 
#Decrease in Urban 
Area_DEG = Area_BF[Area_BF["Class_From"].isin(["0","Urban"]) & (Area_BF["Class_To"].isin(["1","2","4","5",'Forest','Water','Cultivated','Bare']))] 
#Filtering Afforestioan 
Area_AFF = Area_BF[Area_BF["Class_To"].isin(["1","Forest"]) & (Area_BF["Class_From"].isin(["0","2","4","5",'Urban','Water','Cultivated','Bare']))] 
#changed Area 
#Area_Unch = Area_BF[Area_BF["Class_To"]=="Same"]


In [12]:
Are_Def_Sum = Area_Def.groupby("Buffer_Zone").sum()
Are_Urb = Area_Urb.groupby("Buffer_Zone").sum()
Are_AF_Sum = Area_AFF.groupby("Buffer_Zone").sum()
Area_DEG = Area_DEG.groupby("Buffer_Zone").sum()

In [14]:
#Net Deforestioan 
Area_Net_Def = pd.DataFrame(Are_Def_Sum["Area_KM2"]-Are_AF_Sum["Area_KM2"])
#Urbanization 
Are_Urb2 = pd.DataFrame(Are_Urb["Area_KM2"]- Area_DEG["Area_KM2"])

Are_Urb2["Area_KM2"] = Are_Urb2["Area_KM2"].fillna(Are_Urb['Area_KM2'])
Are_Urb = Are_Urb2
Are_Urb

,Area_KM2
Buffer_Zone,
100,0.226187
200,0.375510
300,0.486976
400,0.563313
500,0.638015
600,0.698598
700,0.734315
800,0.778411
900,0.797757


In [15]:
Are_Def_Sum['Area_Dif_DEF'] = Are_Def_Sum["Area_KM2"].diff()
Are_Urb['Area_Dif_U'] = Are_Urb["Area_KM2"].diff()
Area_Net_Def['Area_Dif_NDEF'] = Area_Net_Def["Area_KM2"].diff()

In [17]:
Are_Urb["Area_Dif_U"] = Are_Urb["Area_Dif_U"].fillna(Are_Urb['Area_KM2'])
Area_Net_Def["Area_Dif_NDEF"] = Area_Net_Def["Area_Dif_NDEF"].fillna(Area_Net_Def['Area_KM2'])
Are_Def_Sum["Area_Dif_DEF"] = Are_Def_Sum["Area_Dif_DEF"].fillna(Are_Def_Sum['Area_KM2'])

In [18]:
Sum_A = pd.DataFrame(data =Area_Net_Def["Area_Dif_NDEF"] )
Sum_A["Area_Dif_DEF"] = Are_Def_Sum["Area_Dif_DEF"] 
Sum_A["Area_Dif_U"] = Are_Urb["Area_Dif_U"] 
#Sum_A["Area_UN"] = Are_Un_Sum["Area_Dif_un"]

Sum_A
Sum_A.to_excel(rf'D:\mskenawi\one drive\OneDrive - NTNU\PhD\LUHP\Summary\Summary of Data\Buffer_Analysis_Res\{OP_N}.xlsx')